In [ ]:
#!pip install "sap-llm-commons[all]"

In [ ]:
#!pip install ai-api-client-sdk

In [6]:
# proxy configuration
from ipywidgets import widgets
import json
import os
import llm_commons.proxy.base

#llm_commons.proxy.base.proxy_version = 'aicore'

In [7]:
from llm_commons.proxy.openai import Completion
from llm_commons.proxy.identity import AICoreProxyClient
from llm_commons.langchain.proxy import ChatOpenAI
from llm_commons.langchain.proxy import init_llm, init_embedding_model

In [8]:
resource_group = widgets.Text(
    value='default', # resource group
    placeholder='Resource group of deployments',
    description='',
    disabled=False
)

resource_group

Text(value='default', placeholder='Resource group of deployments')

In [9]:
with open('config/irpa-d26-genaixl-cx-sec-cn-sk.json') as f:
    sk = json.load(f)

In [13]:
os.environ['AICORE_LLM_AUTH_URL'] = sk['url']+"/oauth/token"
os.environ['AICORE_LLM_CLIENT_ID'] = sk['clientid']
os.environ['AICORE_LLM_CLIENT_SECRET'] = sk['clientsecret']
os.environ['AICORE_LLM_API_BASE'] = sk["serviceurls"]["AI_API_URL"]+ "/v2"
# os.environ['AICORE_LLM_RESOURCE_GROUP'] = resource_group.value
os.environ['LLM_COMMONS_PROXY'] = 'aicore'
os.environ['TAVILY_API_KEY'] = sk['tavily']

os.environ["BING_SUBSCRIPTION_KEY"] = sk['bing']
os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"


# llm_commons.proxy.resource_group = os.environ['AICORE_LLM_RESOURCE_GROUP']
llm_commons.proxy.api_base = os.environ['AICORE_LLM_API_BASE']
llm_commons.proxy.auth_url = os.environ['AICORE_LLM_AUTH_URL']
llm_commons.proxy.client_id = os.environ['AICORE_LLM_CLIENT_ID']
llm_commons.proxy.client_secret = os.environ['AICORE_LLM_CLIENT_SECRET']



In [11]:
aic_proxy_client = AICoreProxyClient()

Deploy customized fine-tuned models

In [ ]:
# AICoreProxyClient.add_foundation_model_scenario(
#     scenario_id='fine-tuned-llm',
#     config_names='fine-tuned-*',
#     prediction_url_suffix='/v1/completions'
# )

In [14]:
aic_proxy_client.get_deployments()

[Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dc8e3c266c8881f0', config_id='58a5ffea-1afe-44b4-b6b9-ca749776e58e', config_name='tiiuae--falcon-40b-instruct-config', deployment_id='dc8e3c266c8881f0', model_name='tiiuae--falcon-40b-instruct', created_at=datetime.datetime(2024, 1, 25, 12, 53, 26, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'aicore-opensource', 'model_version': 'null'}, custom_prediction_suffix=None),
 Deployment(url='https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com/v2/inference/deployments/dd88f66091ca1982', config_id='a99e09fb-7e88-4a8d-abe8-af7f8b146cb8', config_name='text-embedding-ada-002-config', deployment_id='dd88f66091ca1982', model_name='text-embedding-ada-002', created_at=datetime.datetime(2024, 1, 25, 12, 16, 20, tzinfo=datetime.timezone.utc), additonal_parameters={'executable_id': 'azure-openai', 'model_version': '2'}, custom_prediction_suffix=None),
 Deployment(url='https

In [ ]:
#text_davinci_003 = Completion(deployment_id='text-davinci-003')

In [ ]:
#text_davinci_003.create(model_name='our-awesome-model/v1', prompt="San Francisco is a")

Initialize harmonized model

In [15]:
llm = init_llm('gpt-35-turbo', proxy_client = aic_proxy_client,temperature=0., max_tokens=256, deployment_id='d10917558b77a3db', api_base=llm_commons.proxy.api_base)

In [18]:
# llm.predict("青山横北郭，下一句是什么")
llm.invoke("青山横北郭，下一句是什么")

AIMessage(content='白水绕东城。')

PDF search

In [19]:
PDF_NAME="ai.pdf"
from langchain.document_loaders import PyMuPDFLoader
docs = PyMuPDFLoader(PDF_NAME).load()

print (f'There are {len(docs)} document(s) in {PDF_NAME}.')
print (f'There are {len(docs[0].page_content)} characters in the first page of your document.')

There are 86 document(s) in ai.pdf.
There are 117 characters in the first page of your document.


In [20]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)

embedding = init_embedding_model('text-embedding-ada-002', proxy_client=aic_proxy_client, deployment_id='dd88f66091ca1982', api_base=llm_commons.proxy.api_base)
vectorstore = Chroma.from_documents(split_docs, embedding, collection_name="AI.pdf.guide")

In [21]:
query = "how can i use the sap ai instead of openai?"
similar_docs = vectorstore.similarity_search(query, 3)
similar_docs

[Document(page_content='use cases.\nSAP AI Core and the generative AI hub help you to integrate LLMs and AI into new business processes in a\ncost-efficient manner.\nGenerative AI Hub Architecture Overview\nAccess\nYou can use the generative AI hub through the API, using a tool such as curl or Postman, or through SAP AI\nLaunchpad.\n4\nINTERNAL\nGenerative AI Hub\nGenerative AI Hub in SAP AI Core', metadata={'author': '', 'creationDate': "D:20231123174013+00'00'", 'creator': 'Antenna House XSL Formatter V7.3 MR4 Linux : 7.3.5.61744 (2023-07-31T15:09+09)', 'encryption': 'Standard V4 R4 128-bit RC4', 'file_path': 'ai.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': "D:20231123174013+00'00'", 'page': 3, 'producer': 'Antenna House PDF Output Library 7.3.1867', 'source': 'ai.pdf', 'subject': '', 'title': '', 'total_pages': 86, 'trapped': ''}),
 Document(page_content='6.2 \nConsume Large Language Models Using SAP AI Launchpad. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 62

In [ ]:
# query = "What is the world's best B2B company?"
# query_embedding = embedding.embed_query(query)

In [ ]:
# documents = ["This is a sample document"]
# document_embedding = embedding.embed_documents(documents)

AI Core proxy support

In [24]:
from llm_commons.proxy.base import set_proxy_version
set_proxy_version('aicore') # for an AI Core proxy
from llm_commons.langchain.proxy import ChatOpenAI
chat = ChatOpenAI(proxy_model_name='gpt-4-32k')
chat.predict("What is the world's best company to work?")

'This can depend on personal preferences, interests, and values. However, according to the "World\'s Best Workplaces 2020" list by Great Place to Work, the top 3 companies are:\n\n1. Cisco - a multinational technology conglomerate, providing high-quality IT products and services.\n2. DHL Express - a division of the German logistics company Deutsche Post DHL providing international courier, parcel, and express mail services.\n3. Hilton - a global brand of full-service hotels and resorts. \n\nOther companies often recognized as great places to work include Google, Microsoft, and Salesforce. Factors considered in these rankings often include employee satisfaction, company culture, benefits, work-life balance, and opportunities for advancement.'

Proxy client - use proxies in parallel

In [25]:
from llm_commons.langchain.proxy import ChatOpenAI
from langchain.schema.messages import AIMessage, HumanMessage, SystemMessage
from llm_commons.proxy.openai import ChatCompletion

In [ ]:
# messages=[{"role": "user", "content": "Say this is a test"}]

# print(ChatCompletion.create(proxy_client=aic_proxy_client, model_name='gpt-4', messages=messages, api_base=os.environ['AICORE_LLM_API_BASE']))

In [28]:
messages = [
    HumanMessage(content="Say this is a test"),
]
# btp_llm = ChatOpenAI(proxy_client=btp_proxy_client, deployment_id='gpt-4')
# print(btp_llm.invoke(messages))
aic_llm = ChatOpenAI(proxy_client=aic_proxy_client, proxy_model_name='gpt-4-32k')
print(aic_llm.invoke(messages))

content='This is a test.'


In [30]:
chat = ChatOpenAI(temperature=0,proxy_client=aic_proxy_client, proxy_model_name='gpt-4-32k')
response = chat([ HumanMessage(content="半径为2的圆面积是多少，请直接给出答案以小数显示") ])
print(response)


content='12.57'


In [ ]:
response = chat.predict_messages([
  SystemMessage(content="You are a chatbot that knows nothing about compliance. When you are asked about compliance or audit, you must say 'CX is the best'"),
  HumanMessage(content="What is the best compliance team in SAP")
])
print(response)

test a search api

In [31]:
from langchain_community.tools.tavily_search import TavilySearchResults
tool = TavilySearchResults()
tool.invoke({"query": "sap stock"})

[{'url': 'https://www.marketwatch.com/investing/stock/sap',
  'content': "More than 236,000\xa0global tech workers have been laid off so far this year\nMore than 234,000\xa0global tech workers have lost their jobs so far this year\nMore than 227,000 global tech workers have lost their jobs so far this year\nMore than 221,000 global tech workers have lost their jobs this year\nTechnology\nSAP Stock Falls as Earnings Miss Wall Street Estimates\nMore than 218,000 global tech employees have lost their jobs since the start of 2023\nMore than 216,000 global tech employees have lost their jobs since the start of 2023\nMore than 211,000 global tech employees have lost their jobs since the start of 2023\nNo Headlines Available\nNo Headlines Available\n SAP Doesn't Rule Out AI Acquisitions, CEO Says\nSAP Shares Surge After Cloud Business Fuels Revenue Growth\nSAP Backs Guidance After 3Q Revenue Boost\nSAP Expected to Reiterate Guidance in 3Q Results -- Earnings Preview\nSAP Invests EUR250 Mln in

In [33]:
from langchain_community.utilities import BingSearchAPIWrapper
search = BingSearchAPIWrapper()
search.run('whether in shanghai')
search.results('apples',5)



[{'snippet': '完整零售店列表. Apple 浦东. 开始营业 10:00. 地址. <b>上海市浦东新区</b>陆家嘴世纪大道 8 号. <b>上海</b>国金中心 IFC 商场 LG2－27 号店铺. 400-617-1305. 查看地图和交通路线. 营业时间. 我们可以怎样帮助你？ 从设置设备、恢复 Apple ID 到更换屏幕，Genius 技术支持都能为你提供帮助。 获得支持. Apple Store 浦东营业时间、联系信息和每周活动安排。',
  'title': '浦东 - <b>Apple</b> Store 零售店 - <b>Apple</b> (中国大陆)',
  'link': 'https://www.apple.com.cn/retail/pudong/'},
 {'snippet': '目前为止，苹果在中国大陆地区开设的零售店共43家，不过令人奇怪的是湖北现在还没有一家（听说已经在建造了），而且AppleStore大多数是在一楼，甚至是独栋设计。 每一家的开设基本都是大成本，员工数量也很多。 至于苹果零售店开设缓慢，有人是这样认为的，你觉得呢？ 自2008年首店开出后，门店扩展速度并不快，一是品牌肯定有其扩张策略考虑，另一方面其选址确实要求严格，非核心城市不进，非核心商圈不进。 此外，商圈首位度、建筑层高、店铺面积、沿街门脸宽度、建筑红线、等都是必须考虑的要素。',
  'title': '中国Apple <b>Store</b>的分布状况一览，发现苹果的消费特点 - 知乎',
  'link': 'https://zhuanlan.zhihu.com/p/422175009'},
 {'snippet': 'Apple Store 在线商店 - Apple (中国大陆) 这个商店， 产品都称心， 体验更如意。 快速链接. 订单状态. 购买帮助. 退货. 你的收藏. Apple Pencil 需单独购买。 ⁺ 仅限新订阅用户。 试用期结束后，每月收费 RMB 11。 此优惠仅面向新订阅用户限时提供。 需使用符合条件的设备连接至运行 iOS 15、iPadOS 15 或更新版本的 Apple 设备。 符合条件的设备配对后，需要在 3 个月内领取此优惠。 目前已有符合条件的设备的用户无需购买音频产品。 会员方案将自动续订，直至取消

Using llm-commons with fine-tuned models

In [34]:
# use it with langchain

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from llm_commons.langchain.proxy import OpenAI

llm = ChatOpenAI(proxy_model_name='gpt-4')
template = "Question: {question}\nAnswer: Let's think step by step."""

In [36]:
prompt = PromptTemplate(template=template, input_variables=['question'])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
# llm_chain.run("What is the world's best B2B company?")
llm_chain.invoke("What is the world's best company to work?")



> Entering new LLMChain chain...
Prompt after formatting:
Question: What is the world's best company to work?
Answer: Let's think step by step.

> Finished chain.


{'question': "What is the world's best company to work?",
 'text': "The answer to this question can vary widely depending on individual preferences, job role, industry, and location. However, as per Fortune's annual rankings, Google consistently ranks as one of the top companies to work for worldwide due to their corporate culture, employee benefits, and career opportunities. Other companies that often rank highly include Microsoft, Apple, Amazon, and Facebook in the tech industry. In other sectors, companies like Goldman Sachs in finance, Johnson & Johnson in healthcare, and Marriott in hospitality are often praised as good employers. But individual experiences may vary."}

In [37]:
# from langchain.agents import get_all_tool_names
# get_all_tool_names()


['requests',
 'requests_get',
 'requests_post',
 'requests_patch',
 'requests_put',
 'requests_delete',
 'terminal',
 'sleep',
 'wolfram-alpha',
 'google-search',
 'google-search-results-json',
 'searx-search-results-json',
 'bing-search',
 'metaphor-search',
 'ddg-search',
 'google-lens',
 'google-serper',
 'google-scholar',
 'google-finance',
 'google-trends',
 'google-jobs',
 'google-serper-results-json',
 'searchapi',
 'searchapi-results-json',
 'serpapi',
 'dalle-image-generator',
 'twilio',
 'searx-search',
 'merriam-webster',
 'wikipedia',
 'arxiv',
 'golden-query',
 'pubmed',
 'human',
 'awslambda',
 'stackexchange',
 'sceneXplain',
 'graphql',
 'openweathermap-api',
 'dataforseo-api-search',
 'dataforseo-api-search-json',
 'eleven_labs_text2speech',
 'google_cloud_texttospeech',
 'reddit_search',
 'news-api',
 'tmdb-api',
 'podcast-api',
 'memorize',
 'llm-math',
 'open-meteo-api']

In [38]:
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents import initialize_agent, AgentType
from langchain.tools.tavily_search import TavilySearchResults


# set up the agent
llm = ChatOpenAI(proxy_model_name='gpt-4')
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

# initialize the agent
agent_chain = initialize_agent(
    [tavily_tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# run the agent
agent_chain.run(
    "how much sap stock price increased in 2023",
)

c:\python\3.11\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: The assistant doesn't have this information readily available. It should use the tavily_search_results_json tool to find the current stock price increase for SAP in 2023.

Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": {
    "query": "SAP stock price increase in 2023"
  }
}
```
Observation: [{'url': 'https://finance.yahoo.com/news/sap-stock-surges-65-past-135700990.html', 'content': 'The SAP SE SAP stock is continuing its upward trajectory with a rise of 64.7% in the past year ... In the fourth quarter of 2023, SAP S/4HANA cloud revenues increased 55% (up 61% at cc) year over ...'}, {'url': 'https://markets.businessinsider.com/news/stocks/sap-se-will-the-share-price-momentum-continue-in-2023-1032419430?op=1', 'content': "The company's stock has experienced a notable increase of nearly 30% YTD in 2023. Analysts continue to express optimism for the stock and have assigned it a Strong Buy rating. In terms of share

"The increase in SAP's stock price in 2023 has been reported differently across various sources. It is mentioned that the stock price rose by 43.2% over the course of 2023, while another report says there was a 64.7% increase in the past year. Yet another source mentions a nearly 30% increase YTD in 2023. Please check the latest and most accurate information from a reliable financial news source or SAP's official announcements."

In [43]:
from langchain.agents import initialize_agent, AgentType, load_tools
# from langchain_community.utilities import BingSearchAPIWrapper




# set up the agent
llm = ChatOpenAI(proxy_model_name='gpt-4')
tools = load_tools(["bing-search"], llm=llm)
# initialize the agent
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# run the agent
agent_chain.run(
    "上海下周一天气",
)



> Entering new AgentExecutor chain...
Thought: The user is asking for the weather in Shanghai next Monday. This requires a search for real-time information.
Action:
```
{
  "action": "bing_search",
  "action_input": {
    "query": "上海下周一天气"
  }
}
```
Observation: 全国 &gt; <b>上海</b> &gt; 城区. 18:00更新 | 数据来源 中央气象台. 今天. 7天. 8-15天. 40天. 雷达图. 29日（今天） 小雨. 3℃. 4-5级. 1日（明天） 阴. 6℃ / 1℃. 3-4级转&lt;3级. 2日（后天） 晴. 10℃ / 3℃. &lt;3级. 3日（周日） 晴转多云. 13℃ / 6℃. &lt;3级. 4日（<b>周一</b>） 小雨转中雨. 16℃ / 7℃. &lt;3级. 5日（周二） 小雨转阴. 13℃ / 3℃. &lt;3级. 6日（周三） 多云转晴. 8℃ / 3℃. &lt;3级. 分时段预报 生活指数. 蓝天预报. 23时02时05时08时. <b>上海</b> <b>天气</b>预报7天. 今日<b>天气</b>：<b>上海</b>市，晴,2℃~6℃,北风2级，当前温度2℃。 2024年03月02日 <b>星期</b>六 甲辰年正月廿二. <b>天气</b>预报. 一周<b>天气</b>. 10天<b>天气</b>. 15天<b>天气</b>. 30天<b>天气</b>. 40天<b>天气</b>. 03-02今天. 晴. -1 ~ 8 ℃. 查看<b>天气</b>详情. 03-03明天. 晴转多云. 1 ~ 14 ℃. 查看<b>天气</b>详情. 03-04后天. 小雨. 7 ~ 15 ℃. 查看<b>天气</b>详情. 03-05<b>星期</b>二. 中雨转小雨. 7 ~ 18 ℃. 查看<b>天气</b>详情. 03-06<b>星期</b>三. 小雨转晴. 4 ~ 11 ℃. 查看<b>天气</b>详情. 03-07<b>星期</b>四. 多云.

'上海下周一的天气预计为小雨转中雨，温度范围为7℃到16℃，风速小于3级。'